### План действий

- [Загрузить и подготовить данные.](#abcd)
- [Исследовать баланс классов, обучить модель без учёта дисбаланса.](#abcde) 
- [Улучшить качество модели, учитывая дисбаланс классов. Обучить разные модели и найти лучшую.](#abcdef)  
- [Провести финальное тестирование.](#abcdefg)

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

# Подготовка данных <a name="abcd"></a>

In [3]:
df = pd.read_csv('/datasets/Churn.csv')
display(df)
df.info()
print(df[df==0].count())
print()
print(df.duplicated().sum())
(10000-9091)/10000*100

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB
RowNumber             0
CustomerId            0
Surname               0
CreditScore           0
Geography             0
Gender                0
Age                   0
Tenure              382
Balance            3617
NumOfProducts         0
HasCrCard          294

9.09

Загрузила и изучила данные. Имеются пропуски в столбце Tenure, 9.09%. В датасете достаточно много нулей, а именно в столбцах Tenure, Balance, HasCrCard, IsActiveMember, Exited. Грубых дубликатов нет. По типам данных - типы соответствуют заявленным признакам. Наименования столбцов представлены разными регистрами. 

In [4]:
df.columns = df.columns.str.lower()
df[df['tenure'].isnull()].head(20)
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 14 columns):
rownumber          9091 non-null int64
customerid         9091 non-null int64
surname            9091 non-null object
creditscore        9091 non-null int64
geography          9091 non-null object
gender             9091 non-null object
age                9091 non-null int64
tenure             9091 non-null float64
balance            9091 non-null float64
numofproducts      9091 non-null int64
hascrcard          9091 non-null int64
isactivemember     9091 non-null int64
estimatedsalary    9091 non-null float64
exited             9091 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.0+ MB


Наименования столбцов привела к нижнему регистру. Проверила строки с пропущенными значениями, данные разнородны, их менее 10%, соответственно удаление строк с пропусками должно пройти безболезненно. Относительно нулей, то их наличие по текущим показателям вполне обоснованно, кол-во недвижимости - недвижимости у клиента может действительно не быть, баланс вполне может быть нулевым, а что касается признаков наличие кр.карты, активность и выход, то данные признаки содержат ответ да/нет(1/0) и поэтому нули здесь очевидны. Данные обработаны. 

# Исследование задачи <a name="abcde"></a>

In [5]:
print(df['exited'].value_counts(normalize=True))

0    0.796062
1    0.203938
Name: exited, dtype: float64


Проверила соотношение классов целевого признака, 2:8(положительный:отрицательный). Наблюдается сильный дисбаланс классов и это может очень плохо сказаться на обучении модели.  

In [6]:
data = df.drop(['surname', 'rownumber', 'customerid'], axis=1)
data_ohe = pd.get_dummies(data, drop_first=True)

target = data_ohe['exited']
features = data_ohe.drop('exited', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

numeric = ['creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'estimatedsalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [7]:
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, predicted_valid)
    print(accuracy_valid)

0.7926292629262927
0.8305830583058306
0.8333333333333334
0.8509350935093509
0.8575357535753575


In [8]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8471027524725867

In [9]:
best_model = None
best_result = 0
for est in range(1, 60):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) 
    model.fit(features_train, target_train) 
    result = model.score(features_valid, target_valid) 
    if result > best_result:
        best_model = model
        best_result = result

print(best_result)
best_model

0.863036303630363


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=43,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

In [10]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8572416002002735

In [11]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
accuracy_valid = accuracy_score(target_valid, predicted_valid)
print(accuracy_valid)

0.8085808580858086


In [12]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.7736191158144302

In [13]:
target_pred_constant = pd.Series(0, index=target.index)
print(accuracy_score(target, target_pred_constant))

0.7960620393796062


Убрала признаки, которые не несут смысловой нагрузки для цели исследования: фамилия, индекс строки в данных, уникальный идентификатор клиенты. Для признаков классификации пол и география применила технику прямого кодирования, получив по текущим признаками дамми-признаки. Обучила 3 вида моделей с учетом дисбаланса классов. Что и требовалось доказать: модели с таким жестким дисбалансом классов работают практически также, как и константная модель, которая каждому объекту прогнозирует класс "0". Проверку на адекватность модели(при дисбалансе классов) не прошли. Относительно подбора гиперпараметров следует сказать, что модель решающего дерева показывает наилучшее значение accuracy при глубине 6. Лучшая модель случайного леса с кол-вом деревьев = 43.    

# Борьба с дисбалансом <a name="abcdef"></a>

In [14]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.509731232622799


In [15]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.7777884132187896

In [16]:
model = DecisionTreeClassifier(random_state=12345, class_weight='balanced', max_depth=6)
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5630885122410546


In [17]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8320960429410021

In [18]:
model = RandomForestClassifier(random_state=12345, n_estimators=43) 
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5964343598055105


In [19]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8534533379229351

Модели логистической регрессии, дерева решений и случайного леса, борьба с дисбалансом при помощи взвешивания классов.

In [20]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = LogisticRegression(solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5085972850678734


In [21]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.7779006989325494

In [22]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6)
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5630885122410546


In [23]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8321227338073877

In [24]:
model = RandomForestClassifier(random_state=12345, n_estimators=43) 
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.6115942028985507


In [25]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8532278461207127

Модели логистической регрессии, дерева решений и случайного леса, борьба с дисбалансом при помощи увеличения выборки.

In [26]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(features_train, target_train, 0.25)

model = LogisticRegression(solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)

print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5044883303411131


In [27]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.7770355467117773

In [28]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6)
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5636704119850188


In [29]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8343500405885245

In [30]:
model = RandomForestClassifier(random_state=12345, n_estimators=43) 
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.577560975609756


In [31]:
probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8572995838065594

Модели логистической регрессии, дерева решений и случайного леса, борьба с дисбалансом при помощи уменьшения выборки.

Лучший результат f1-меры=0.61 выдает случайный лес при увеличении выборки.

# Тестирование модели <a name="abcdefg"></a>

In [32]:
target = data_ohe['exited']
features = data_ohe.drop('exited', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345)

numeric = ['creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'estimatedsalary']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

In [33]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

model = RandomForestClassifier(random_state=12345, n_estimators=43) 
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid)
print("F1:", f1_score(target_valid, predicted_valid))

probabilities_valid = model.predict_proba(features_valid)
probabilities_one_valid = probabilities_valid[:, 1]

auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
print("AUC:", auc_roc)

F1: 0.6115942028985507
AUC: 0.8532278461207127


проверка на валидационной выборке

In [34]:
model = RandomForestClassifier(random_state=12345, n_estimators=43) 
model.fit(features_upsampled, target_upsampled)
predicted_test = model.predict(features_test)
print("F1:", f1_score(target_test, predicted_test))

probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]

auc_roc = roc_auc_score(target_test, probabilities_one_test)
print("AUC:", auc_roc)

F1: 0.6070878274268104
AUC: 0.8425062297678101


проверка на тестовой выборке

Провела финальное тестирование модели случайного леса на валидационной и тестовой выборках, достигла целевого результата f1-меры выше 0.59.

Метрики на необходимом уровне, что говорит о качественной работе модели. Т.о. модель случайного леса с кол-вом деревьев 43 готова к работе над текущей задачей.   